# Analysis of probe measurements

## Setting up Python

Those Python libraries need to be imported before analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statistics
from ipywidgets import *

## Importing the data

The data files are saved as csv and imported in here

In [2]:
hmds = pd.read_csv("Data/hmds_byloc.csv",na_values=['NC']) #hmds-treated sample
gl = pd.read_csv("Data/gl_byloc.csv",na_values=['NC'])     #large gold sample
gs = pd.read_csv("Data/gs_byloc.csv",na_values=['NC'])      #small gold sample
boot = pd.read_csv("Data/boot_byloc.csv",na_values=['no','nan'])  #reference sample

An imported table looks like this, with the location referring to the pair of electrode measured

In [3]:
boot

,Location,before,200,300,400,500
0,A1-B1,7.68,30.00,27.1,NaN,NaN
1,A1-A2,5.65,11.68,10.2,11.0,NaN
2,A1-B2,2.30,18.97,16.9,NaN,NaN
3,B1-B2,5.21,11.03,10.5,NaN,NaN
4,A2-B2,5.04,19.80,17.4,NaN,NaN
5,A2-B1,11.63,31.00,27.5,NaN,NaN


Each of the sample have their own size. Hmds was a 3x4, gl a 3x3, gs a 2x2, and boot a 2x2. The diagrams of the samples are shown below.

%%html
<img src="Figures/hmds.png" width="150" height="150" align="center"/>
<img src="Figures/gl.png" width="150" height="150" align="center"/>
<img src="Figures/gs.png" width="150" height="150" align="center"/>
<img src="Figures/gs.png" width="150" height="150" align="center"/>

For analysis the sample data is linked to the shape of the samples by using a Python dictionary

In [4]:
exps = {'hmds': (hmds, 'D', 3), 'gl': (gl, 'C', 3), 'gs': (gs, 'B', 2), 'boot': (boot, 'B', 2)}

## Useful Python functions for analysis

### Line extractor
First thing to do is a function that returns the temperature as x and the resistance as y when given the experiment and the location

In [5]:
def get_temperature_resistance_by_loc(exp, loc): #inputs are the pandas dataframe and a string with the location
    x = []
    y = []
    df = exp
    initial_row = '200'
    row = df[df["Location"] == loc]
    if row.empty:
        return x,y
    if not row[initial_row].isnull().item():
        x.append(initial_row)
        y.append(float(row[initial_row].item()))
    for i in df.columns[3:]:
        if not row[initial_row].isnull().item():
            x.append(i)
            y.append(float(row[i].item()))
    return x,y

These can be plotted directly

In [25]:
%matplotlib widget

# making the input widgets
sample_list = [('HMDS-treated','hmds'), ('Gold large','gl'), ('Gold small','gs'), ('Reference','boot')]
sample_text = widgets.Dropdown(description="Sample", options=sample_list, width=100)
location_list = list(hmds['Location'])
location_text = widgets.Dropdown(description="Location", options=location_list, width=100)
log_check = widgets.Checkbox(value=False, description='Log scale')

#function to update possible locations in dropdown from samples dropdown
def update_loc(_):
    sample = exps[sample_text.value][0]
    location_text.options = list(sample['Location'])
sample_text.observe(update_loc)

# display all
text_container = widgets.HBox([sample_text, location_text, log_check])
display(text_container)

#function to receive location input
def on_loc_selected(_):
    sample = exps[sample_text.value][0]
    loc = location_text.value
    log = log_check.value
    set_plot(sample, loc, log)
location_text.observe(on_loc_selected)
log_check.observe(on_loc_selected)

#function to update plot
def set_plot(sample=hmds, loc='A1-B1', log=False):
    x,y = get_temperature_resistance_by_loc(sample,loc)
    ax.clear()
    ax.set_ylabel("Resistance (kΩ)")
    ax.set_xlabel("Annealing temperature (°C)")
    ax.plot(x,y, label=loc)
    ax.legend()
    if log:
        ax.set_yscale('log')
    fig.canvas.draw()

fig = plt.figure()
ax = fig.add_subplot(111)
set_plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Device extractor
As my analysis is focused on the statistics I only look at the data for close pairs of devices (~same distance apart for all samples). This can be done with a function that returns the list of all possible locations for each sample.

In [7]:
def get_devices(lmax, nmax):
    devices = []
    ls = ['A', 'B', 'C', 'D']
    for letter in ls[0:ls.index(lmax)+1]:
        for number in range(1, nmax):
            device = f"{letter}{number}-{letter}{number+1}"
            devices.append(device)
    for number in range(1, nmax+1):
        for i,letter in enumerate(ls[0:ls.index(lmax)]):
            device = f"{letter}{number}-{ls[i+1]}{number}"
            devices.append(device)
    return devices

For example, for the gold sample, the length goes to letter 'C', and width to number 3

In [8]:
get_devices('C',3)

['A1-A2',
 'A2-A3',
 'B1-B2',
 'B2-B3',
 'C1-C2',
 'C2-C3',
 'A1-B1',
 'B1-C1',
 'A2-B2',
 'B2-C2',
 'A3-B3',
 'B3-C3']

### Resistance extractor
Using the function to extract the devices, we can now get the resistances for each of these locations for a given sample and at a given temperature

In [9]:
def get_resistances(exp, temp):
    rlist = []
    devices = get_devices(exp[1], exp[2])
    for device in devices:
        r = exp[0][exp[0]["Location"] == device][str(temp)]
        try:
            rlist.append(float(r))
        except:
            rlist.append('NA')
            pass
    return rlist

For example the small gold sample at 500 °C

In [10]:
get_resistances(exps['gs'], 500)

[1.51, 4.52, 6.68, 4.56]

### Percent increase extractor
Another thing we might want to calculate is the percent increase in each of the devices between two temperatures.

In [26]:
def get_percent_increase(exp, t1, t2):
    r1s = get_resistances(exp, t1)
    r2s = get_resistances(exp, t2)
    changes = [(r2-r1)/r1 for (r1, r2) in zip(r1s, r2s)]
    return changes

For example the small gold sample between 300 °C and 500 °C

In [27]:
get_percent_increase(exps['gs'],300,500)

[-0.2705314009661835,
 0.4169278996865203,
 0.4746136865342162,
 0.5099337748344369]

## The plots

Starting with the plot used in the presentation as I think it gives the best overview

In [28]:
X = [300, 400, 500, 600, 700]
samples = ['hmds', 'gs', 'gl']
Y = {}
top_y = 85
for sample in samples:
    Y[sample] = {'t':[[],[]], 'mean':[[],[]], 'dev':[[], []], 'min':[[], []], 'max':[[], []]}
for t in X:
    for sample in samples:
        change_list = get_percent_increase(exps[sample], 300, t)
        y = statistics.mean(change_list)
        if y*100 < top_y: i = 0
        else: i = 1
        Y[sample]['t'][i].append(t)
        Y[sample]['mean'][i].append(y*100)
        Y[sample]['dev'][i].append(statistics.pstdev(change_list)*100)
        Y[sample]['min'][i].append(abs(y-np.percentile(change_list,20))*100)
        Y[sample]['max'][i].append(abs(np.percentile(change_list,80)-y)*100)
#for sample in samples:
#    for attr in list(Y[sample].keys()):  
#        Y[sample][attr][0].append(Y[sample][attr][1][0])
#        if sample != 'gs':
#            Y[sample][attr][1] = [Y[sample][attr][0][-2]] + Y[sample][attr][1]
    
# Defining the figure and figure size
#fig, ax = plt.subplots(figsize=(12, 8))
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(9,7), dpi=60)
top_y = 85

# Plotting the error bars
def length(y_sample):
    return len([y for y in y_sample if y<top_y])
for i, ax in enumerate([ax2, ax1]):
    cut = lambda pts:{'hmds':[pts[:length(Y_hmds)+1], pts[length(Y_hmds)-1:]], 
                      'gs':[pts[:length(Y_gs)+1], pts[length(Y_gs):]],
                      'gl':[pts[:length(Y_gl)+1], pts[length(Y_gl):]]}
    ax.errorbar([x-9 for x in Y['hmds']['t'][i]], y=Y['hmds']['mean'][i], 
                yerr=[Y['hmds']['min'][i], Y['hmds']['max'][i]], 
                fmt='o-', ecolor='dimgrey', color='royalblue', capsize=5, 
                label='HMDS               (batch 1)', markersize=15, lw=2)
    ax.errorbar([x+9 for x in Y['gl']['t'][i]], y=Y['gl']['mean'][i], 
                yerr=[Y['gl']['min'][i], Y['gl']['max'][i]], 
                fmt='^-', ecolor='dimgrey', color='orangered', capsize=5, 
                label='2 nm gold        (batch 1)', markersize=15, lw=2)
    ax.errorbar([x for x in Y['gs']['t'][i]], Y['gs']['mean'][i], 
                yerr=[Y['gs']['min'][i], Y['gs']['max'][i]], 
                fmt='s-', ecolor='dimgrey', color='firebrick', capsize=5, 
                label='2 nm gold        (batch 2)', markersize=15, lw=2)

#Zoom in / limit views
ax1.set_ylim(top_y-10, 20000)  # outliers only
ax1.set_yscale('log')
ax1.set_yticks([200, 500, 1000, 2000, 5000, 10000])
ax1.set_yticklabels([200, 500, 1000, 2000, 5000, 10000])
ax2.set_ylim(-50, top_y+10)  # most of the data

#add the lines
ax1.plot([Y['hmds']['t'][1][0]-9, 630], [Y['hmds']['mean'][1][0], 85], '-', color='royalblue', lw=2)
ax2.plot([Y['hmds']['t'][0][-1]-9, 610], [Y['hmds']['mean'][0][-1], 85], '-', color='royalblue', lw=2)
ax1.plot([Y['gl']['t'][1][0]+9, 489], [Y['gl']['mean'][1][0], 85], '-', color='orangered', lw=2)
ax2.plot([Y['gl']['t'][0][-1]+9, 479], [Y['gl']['mean'][0][-1], 85], '-', color='orangered', lw=2)
ax1.plot([Y['gs']['t'][1][0], 572], [Y['gs']['mean'][1][0], 85], '-', color='firebrick', lw=2)
ax2.plot([Y['gs']['t'][0][-1], 555], [Y['gs']['mean'][0][-1], 85], '-', color='firebrick', lw=2)

# hide the spines between ax and ax2
ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop='off')  # don't put tick labels at the top
ax2.xaxis.tick_bottom()
f.tight_layout()

d = .02  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them
kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
ax1.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
ax1.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal
    
#Adding reference information
bbox = dict(boxstyle="round, pad=0.3", fc="lemonchiffon", ec="gray", lw=1)
arrowprops = dict(
    arrowstyle = "->")#,
    #connectionstyle = "angle,angleA=-10,angleB=-80,rad=-20")
ax2.annotate('Reference sample\nno longer conductive', xy=(400, -50), xytext=(550, -20),
           bbox=bbox, arrowprops=arrowprops, fontsize=17)

# Adding plotting parameters
#ax.set_yscale('log')
ax2.set_xlabel('Annealing temperature (°C)', fontsize=19, labelpad=2)
#ax1.set_ylabel('Increase in resistance (%)', fontsize=19)
ax1.yaxis.set_label_position('left') 
#ax.set_ylim(-40, 300)
ax1.legend(loc='best', frameon=True, fontsize=17)
ax2.tick_params(labelsize=17)
ax1.tick_params(labelsize=17)
f.text(-0.05, 0.2, 'Increase in resistance (%)', fontsize=19, rotation='vertical')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(-0.05, 0.2, 'Increase in resistance (%)')